# Build a song recommender system

In [4]:
import turicreate

# Load some music data

In [6]:
song_data = turicreate.SFrame('song_data.sframe/')

# Explore our data

In [7]:
song_data

user_id,song_id,listen_count,title,artist
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOAKIMP12A8C130995,1,The Cove,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBBMDR12A8C13253B,2,Entre Dos Aguas,Paco De Lucia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBXHDL12A81C204C0,1,Stronger,Kanye West
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOBYHAJ12A6701BF1D,1,Constellations,Jack Johnson
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODACBL12A8C13C273,1,Learn To Fly,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODDNQT12A6D4F5F7E,5,Apuesta Por El Rock 'N'Roll ...,Héroes del Silencio
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SODXRTY12AB0180F3B,1,Paper Gangsta,Lady GaGa
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFGUAY12AB017B0A8,1,Stacked Actors,Foo Fighters
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOFRQTD12A81C233C0,1,Sehr kosmisch,Harmonia
b80344d063b5ccb3212f76538f3d9e43d87dca9e ...,SOHQWYZ12A6D4FA701,1,Heaven's gonna burn youreyes ...,Thievery Corporationfeat. Emiliana Torrini ...


## Show the most popular songs in the dataset

In [8]:
song_data['song'].show()

Materializing SArray

# Count the number of unique users in the data

In [9]:
users = song_data['user_id'].unique()

In [10]:
len(users)

66346

# Exploring the Data

In [17]:
# To get the number of unique listeners each of the artists below have had

artists_list = ['Kanye West', 'Foo Fighters', 'Taylor Swift', 'Lady GaGa']

for artist in artists_list:
  print(artist,': ', len(song_data[song_data['artist']==artist]['user_id'].unique()),'\n')


Kanye West :  2522 

Foo Fighters :  2055 

Taylor Swift :  3246 

Lady GaGa :  2928 



In [26]:
# To get the most and least popular artist

popularity = song_data.groupby(key_column_names=['artist'], operations={'total_count': turicreate.aggregate.SUM('listen_count')})

print('The most popular artist is:\n')
print(popularity.sort(['total_count'],ascending=False).head(1))
print('The least popular artist is:\n')
print(popularity.sort(['total_count'],ascending=False).tail(1))

The most popular artist is:

+---------------+-------------+
|     artist    | total_count |
+---------------+-------------+
| Kings Of Leon |    43218    |
+---------------+-------------+
[1 rows x 2 columns]

The least popular artist is:

+-----------------+-------------+
|      artist     | total_count |
+-----------------+-------------+
| William Tabbert |      14     |
+-----------------+-------------+
[1 rows x 2 columns]



# Create a song recommender

In [27]:
train_data,test_data = song_data.random_split(.8,seed=0)

## Create a very simple popularity recommender

In [28]:
popularity_model = turicreate.popularity_recommender.create(train_data,
                                                           user_id = 'user_id',
                                                           item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

Data prepared in: 1.50498s

893580 observations to process; with 9952 unique items.

## Use the popularity model to make some predictions

In [29]:
popularity_model.recommend(users=[users[0]])

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Sehr kosmisch - Harmonia,4754.0,1
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Undo - Björk,4227.0,2
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,You're The One - DwightYoakam ...,3781.0,3
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Dog Days Are Over (RadioEdit) - Florence + The ...,3633.0,4
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Revelry - Kings Of Leon,3527.0,5
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Horn Concerto No. 4 in Eflat K495: II. Romance ...,3161.0,6
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Secrets - OneRepublic,3148.0,7
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Fireflies - CharttraxxKaraoke ...,2532.0,8
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Tive Sim - Cartola,2521.0,9
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Drop The World - LilWayne / Eminem ...,2053.0,10


In [30]:
popularity_model.recommend(users=[users[1]])

user_id,song,score,rank
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Sehr kosmisch - Harmonia,4754.0,1
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Undo - Björk,4227.0,2
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,You're The One - DwightYoakam ...,3781.0,3
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Dog Days Are Over (RadioEdit) - Florence + The ...,3633.0,4
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Revelry - Kings Of Leon,3527.0,5
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Horn Concerto No. 4 in Eflat K495: II. Romance ...,3161.0,6
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Secrets - OneRepublic,3148.0,7
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Hey_ Soul Sister - Train,2538.0,8
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Fireflies - CharttraxxKaraoke ...,2532.0,9
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Tive Sim - Cartola,2521.0,10


# Build a recommender with personalization

In [31]:
personalized_model = turicreate.item_similarity_recommender.create(train_data,
                                                                  user_id = 'user_id',
                                                                  item_id = 'song')

Warning: Ignoring columns song_id, listen_count, title, artist;

To use one of these as a rating column, specify the column name to be used as target

and use a method that allows the use of a target.

Preparing data set.

Data has 893580 observations with 66085 users and 9952 items.

...

| 2.64s                               | 100              | 9952            |

+-------------------------------------+------------------+-----------------+

Finalizing lookup tables.

Generating candidate set for working with new users.

Finished training in 3.74559s

## Apply personalized model to make song recommendations

In [32]:
personalized_model.recommend(users=[users[0]])

user_id,song,score,rank
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Cuando Pase El Temblor -Soda Stereo ...,0.019450453611520622,1
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Fireflies - CharttraxxKaraoke ...,0.014473731701190654,2
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Love Is A Losing Game -Amy Winehouse ...,0.014286596041459303,3
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Marry Me - Train,0.014133471709031325,4
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Secrets - OneRepublic,0.013591665488023024,5
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Sehr kosmisch - Harmonia,0.013398789442502536,6
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Te Hacen Falta Vitaminas- Soda Stereo ...,0.01293028317964994,7
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,OMG - Usher featuringwill.i.am ...,0.012777828253232516,8
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,Y solo se me ocurreamarte (Unplugged) - ...,0.012341127945826603,9
c66c10a9567f0d82ff31441a9fd5063e5cd9dfe8 ...,No Dejes Que... -Caifanes ...,0.012104249917543851,10


In [33]:
personalized_model.recommend(users=[users[1]])

user_id,song,score,rank
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Where The Boat LeavesFrom (Album) - Zac Brown ...,0.06153608858585358,1
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Different Kind Of Fine(Album) - Zac Brown Band ...,0.06052837520837784,2
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Jolene (Album) - ZacBrown Band ...,0.05786820501089096,3
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Sic 'Em On A Chicken(Album) - Zac Brown Band ...,0.05518669635057449,4
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Who's Kissing You Tonight- Jason Aldean ...,0.05306333303451538,5
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,What Country Is - LukeBryan ...,0.037490807473659515,6
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Highway 20 Ride (Album) -Zac Brown Band ...,0.03733157366514206,7
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Do I - Luke Bryan,0.033077359199523926,8
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,One Fine Wire - ColbieCaillat ...,0.03125,9
02f015d32ac2cd1e52d26e3ec36048711dd5711b ...,Midnight Bottle - ColbieCaillat ...,0.030737705528736115,10


# Apply model to find similar songs in the data set

In [34]:
personalized_model.get_similar_items(['With Or Without You - U2'])

song,similar,score,rank
With Or Without You - U2,I Still Haven't FoundWhat I'm Looking For ...,0.04285717010498047,1
With Or Without You - U2,Hold Me_ Thrill Me_ KissMe_ Kill Me - U2 ...,0.033734917640686035,2
With Or Without You - U2,Window In The Skies - U2,0.03283584117889404,3
With Or Without You - U2,Vertigo - U2,0.03007519245147705,4
With Or Without You - U2,Sunday Bloody Sunday - U2,0.02713179588317871,5
With Or Without You - U2,Bad - U2,0.02517986297607422,6
With Or Without You - U2,A Day Without Me - U2,0.023715436458587646,7
With Or Without You - U2,Another Time AnotherPlace - U2 ...,0.020325183868408203,8
With Or Without You - U2,Walk On - U2,0.020202040672302246,9
With Or Without You - U2,Get On Your Boots - U2,0.019685029983520508,10


In [35]:
personalized_model.get_similar_items(['Chan Chan (Live) - Buena Vista Social Club'])

song,similar,score,rank
Chan Chan (Live) - BuenaVista Social Club ...,Murmullo - Buena VistaSocial Club ...,0.1881188154220581,1
Chan Chan (Live) - BuenaVista Social Club ...,La Bayamesa - Buena VistaSocial Club ...,0.1871921420097351,2
Chan Chan (Live) - BuenaVista Social Club ...,Amor de Loca Juventud -Buena Vista Social Club ...,0.18483412265777588,3
Chan Chan (Live) - BuenaVista Social Club ...,Diferente - Gotan Project,0.021459221839904785,4
Chan Chan (Live) - BuenaVista Social Club ...,Mistica - Orishas,0.020576119422912598,5
Chan Chan (Live) - BuenaVista Social Club ...,Hotel California - GipsyKings ...,0.019304990768432617,6
Chan Chan (Live) - BuenaVista Social Club ...,Nací Orishas - Orishas,0.019157111644744873,7
Chan Chan (Live) - BuenaVista Social Club ...,Gitana - Willie Colon,0.018796980381011963,8
Chan Chan (Live) - BuenaVista Social Club ...,Le Moulin - Yann Tiersen,0.018796980381011963,9
Chan Chan (Live) - BuenaVista Social Club ...,Criminal - Gotan Project,0.018779337406158447,10


# Compare the models quantitatively
We now formally compare the popularity and the personalized models using precision-recall curves. 

In [40]:
model_performance = turicreate.recommender.util.compare_models(test_data, [popularity_model, personalized_model], user_sample=.05)

compare_models: using 2931 users to estimate model performance
PROGRESS: Evaluate model M0


recommendations finished on 1000/2931 queries. users per second: 3057.23

recommendations finished on 2000/2931 queries. users per second: 2964.73


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.028317980211531906 | 0.006941841281657043 |
|   2    | 0.025076765609007175 | 0.01237733133843676  |
|   3    | 0.022176731490958737 | 0.01695769165062922  |
|   4    | 0.020982599795291705 | 0.02143041764331427  |
|   5    | 0.01958375980893894  | 0.025194902523459344 |
|   6    | 0.018594336403957683 | 0.02835937992028066  |
|   7    | 0.017985085538821492 | 0.032094370809826286 |
|   8    | 0.017144319344933472 | 0.03424831262599942  |
|   9    | 0.01641457219758143  | 0.037318395429961536 |
|   10   | 0.01586489252814744  | 0.040301833009100146 |
+--------+----------------------+----------------------+
[10 rows x 3 columns]

PROGRESS: Evaluate model M1


recommendations finished on 1000/2931 queries. users per second: 2693.11

recommendations finished on 2000/2931 queries. users per second: 2756.85


Precision and recall summary statistics by cutoff
+--------+----------------------+----------------------+
| cutoff |    mean_precision    |     mean_recall      |
+--------+----------------------+----------------------+
|   1    | 0.10474240873422042  | 0.030710422675622302 |
|   2    | 0.08648925281473903  | 0.04855018318426713  |
|   3    | 0.07449107244398953  | 0.061455118671085926 |
|   4    |  0.0668713749573525  | 0.07307479188236606  |
|   5    | 0.06038894575230299  | 0.08079333949344182  |
|   6    | 0.05578300921187312  | 0.08866066266270972  |
|   7    | 0.05146951308670852  | 0.09584584498862898  |
|   8    | 0.04827703855339475  | 0.10209487951299612  |
|   9    | 0.045642367034383534 | 0.10768906166807916  |
|   10   | 0.04288638689866938  | 0.11149651033223199  |
+--------+----------------------+----------------------+
[10 rows x 3 columns]



The table shows that the personalized model provides much better performance.